In [20]:
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import ( GradientBoostingRegressor, RandomForestRegressor)
from sklearn.metrics import mean_squared_error

In [2]:
!pip install mlflow ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


In [46]:
!pip install optuna optuna-dashboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 11.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
import mlflow
import os

In [4]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

In [ ]:
token = "" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /home/lenovo/.config/ngrok/ngrok.yml                         


In [ ]:
from pyngrok import ngrok
ngrok.connect(5000, "http")

In [16]:
mlflow.set_tracking_uri("http://localhost:5000")

In [18]:
exp_id = mlflow.create_experiment(name="PS_1D", artifact_location="mlruns/")

'2'

In [22]:
df_R1 = pd.read_csv("/content/data_R1.csv")

In [23]:
df_R1[['Sortname','Location']] = df_R1[['Sortname','Location']].astype('category')

In [24]:
df_R1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562 entries, 0 to 1561
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Start Time    1562 non-null   object  
 1   Sortname      1562 non-null   category
 2   Location      1562 non-null   category
 3   CaO_AVG       1562 non-null   float64 
 4   SiO2_AVG      1562 non-null   float64 
 5   Al2O3_AVG     1562 non-null   float64 
 6   Fe2O3_AVG     1562 non-null   float64 
 7   MgO_AVG       1562 non-null   float64 
 8   NO.SO3.P_AVG  1562 non-null   float64 
 9   Na2O.P_AVG    1562 non-null   float64 
 10  K2O.P_AVG     1562 non-null   float64 
 11  TiO2_AVG      1562 non-null   float64 
 12  P2O5_AVG      1562 non-null   float64 
 13  Mn2O3_AVG     1562 non-null   float64 
 14  Cl_AVG        1562 non-null   float64 
 15  Cr2O3_AVG     1562 non-null   float64 
 16  LOI.P_AVG     1562 non-null   float64 
 17  NO.R45_AVG    1562 non-null   float64 
 18  WC_AVG  

In [25]:
features = df_R1.drop(['Start Time','R1'], axis = 1)
labels = df_R1['R1']

In [26]:
features_train, features_test, labels_train, labels_test = train_test_split(
    features,
    labels,
    test_size = 0.2,
    random_state = 100
)

In [27]:
print(features_train.shape)
print(features_test.shape)

(1249, 22)
(313, 22)


In [ ]:
from xgboost import XGBRegressor

def train_model(
    features,
    labels,
    max_depth,
    n_estimators,
    learning_rate,
    random_state,
    enable_categorical
):
    model = XGBRegressor(
        n_estimators = n_estimators,
        max_depth = max_depth,
        learning_rate = learning_rate,
        random_state = random_state,
        enable_categorical =True
    ).fit(features, labels)

    return model

In [ ]:
model = train_model(
    features = features_train,
    labels = labels_train,
    max_depth = 5,
    n_estimators = 100,
    learning_rate = 1e-4,
    random_state = 0,
    enable_categorical =True
)

In [ ]:


preprocessor = make_column_transformer((OneHotEncoder(), ['Sortname', 'Location']), remainder='passthrough' )
pipeline = make_pipeline(preprocessor,  LinearRegression())
pipeline.fit(features_train, labels_train)
y_pred = pipeline.predict(features_test)
score = pipeline.score(features_test, labels_test)
BR_loss = mean_squared_error(labels_test, y_pred)
score, BR_loss

(0.9599954851324098, 1.3222272437971443)

In [ ]:
preprocessor = make_column_transformer((OneHotEncoder(), ['Sortname', 'Location']), remainder='passthrough' )
pipeline = make_pipeline(preprocessor,  GradientBoostingRegressor(random_state=42))
pipeline.fit(features_train, labels_train)
y_pred = pipeline.predict(features_test)
score = pipeline.score(features_test, labels_test)
BR_loss = mean_squared_error(labels_test, y_pred)
score, BR_loss

(0.9628195041496845, 1.2288879071247514)

In [ ]:
preprocessor = make_column_transformer((OneHotEncoder(), ['Sortname', 'Location']), remainder='passthrough' )
pipeline = make_pipeline(preprocessor,  RandomForestRegressor(random_state=42))
pipeline.fit(features_train, labels_train)
y_pred = pipeline.predict(features_test)
score = pipeline.score(features_test, labels_test)
BR_loss = mean_squared_error(labels_test, y_pred)
score, BR_loss

(0.9630105478040647, 1.2225735417234427)

In [32]:
def train_model(features, labels, model):#params

  preprocessor = make_column_transformer((OneHotEncoder(), ['Sortname', 'Location']), remainder='passthrough' )
  pipeline = make_pipeline(preprocessor,  model)
  pipeline.fit(features, labels)

  return pipeline

def eval_model(pipeline, features, labels):
  y_pred = pipeline.predict(features)

  score = pipeline.score(features, labels)
  mae = mean_squared_error(labels, y_pred)
  return score, mae

In [41]:
def mlflow_run(
    features_train,
    labels_train,
    features_test,
    labels_test,
    model,
    #params,
    exp,
    run_name
    ):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    run = mlflow.start_run(run_name=run_name, experiment_id = exp)

    model = train_model(
        features=features_train,
        labels=labels_train,
        model = model,
        #params = params
        )

    score, mae = eval_model(model, features_test, labels_test)

    mlflow.log_params({
      #params
    })

    mlflow.sklearn.log_model(model, "model")
    mlflow.log_param("run_name", run_name)

    mlflow.log_metrics({
        "score": score,
        "mae" : mae })
    mlflow.end_run()

    return run, score, mae

In [49]:
def mlflow_run_2(
    features_train,
    labels_train,
    features_test,
    labels_test,
    model,
    #params,
    exp,
    run_name
    ):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    run = mlflow.start_run(run_name=run_name, experiment_id = exp)

    model = train_model(
        features=features_train,
        labels=labels_train,
        model = model,
        #params = params
        )

    score, mae = eval_model(model, features_test, labels_test)

    mlflow.sklearn.log_model(model, "model")
    mlflow.log_param("run_name", run_name)

    mlflow.log_metrics({
        "score": score,
        "mae" : mae })
    mlflow.end_run()

    return run, score, mae

In [50]:
mlflow.end_run()

In [42]:
model = LinearRegression()

run, score, mae = mlflow_run(
        features_train,
        labels_train,
        features_test,
        labels_test,
        model,
        #params,
        exp=exp_id,
        run_name="Base_LinearRegressor"
        )
print(run.info.run_name)
print(os.listdir(run.info.artifact_uri))
print(score)
print(mae)

Base_LinearRegressor
['model']
0.9599951588516586
1.3222380280123711


In [51]:
def objective(trial):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    max_depth = trial.suggest_int("max_depth", 2, 10)
    n_estimators = trial.suggest_int("n_estimators", 25, 200)
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1, log=True)

    _, score, mae = mlflow_run_2(
        features_train=features_train,
        labels_train=labels_train,
        features_test=features_test,
        labels_test=labels_test,
        model=GradientBoostingRegressor(
            max_depth=max_depth,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            random_state=42,
        ),
        exp=exp_id,
        run_name="optuna",
    )
    return score

In [52]:
import optuna

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///hp.db",
    study_name="PS_1D",
    load_if_exists=True
    )
study.optimize(func=objective, n_trials=30, n_jobs=1)

[I 2023-12-06 03:09:55,251] Using an existing study with name 'PS_1D' instead of creating a new one.
[I 2023-12-06 03:10:02,140] Trial 1 finished with value: 0.9617317890925069 and parameters: {'max_depth': 5, 'n_estimators': 166, 'learning_rate': 0.13815379330653027}. Best is trial 1 with value: 0.9617317890925069.
[I 2023-12-06 03:10:06,325] Trial 2 finished with value: 0.960607762296443 and parameters: {'max_depth': 2, 'n_estimators': 156, 'learning_rate': 0.2880310887502577}. Best is trial 1 with value: 0.9617317890925069.
[I 2023-12-06 03:10:11,603] Trial 3 finished with value: 0.0020557077257901124 and parameters: {'max_depth': 9, 'n_estimators': 103, 'learning_rate': 1.8791773727539863e-05}. Best is trial 1 with value: 0.9617317890925069.
[I 2023-12-06 03:10:17,811] Trial 4 finished with value: 0.9534211083983708 and parameters: {'max_depth': 3, 'n_estimators': 180, 'learning_rate': 0.5576484781936003}. Best is trial 1 with value: 0.9617317890925069.
[I 2023-12-06 03:10:21,414] 